<a href="https://colab.research.google.com/github/gokulanv/LeNet_Pytorch/blob/master/LeNetPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

LeNet - Simple Neural Network

In [53]:
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    #Conv layers
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    # FC Layers
    self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=84)
    self.fc3 = nn.Linear(in_features=84, out_features=10)

  def forward(self, x):
    x = F.max_pool2d(self.conv1(x), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = NeuralNet()
print(net)

NeuralNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [54]:
params = list(net.parameters())
print(params[0].size())

torch.Size([6, 1, 5, 5])


In [55]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0535,  0.1233, -0.0203,  0.0653, -0.0725, -0.0901,  0.0750,  0.0137,
          0.0887, -0.1660]], grad_fn=<AddmmBackward>)


In [56]:
#Loss computation

output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0692, grad_fn=<MseLossBackward>)


In [57]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [58]:
net.zero_grad()

print('conv1 grad before backprop')
print(net.conv1.weight.grad)

loss.backward()

print('conv1 grad after backprop')
print(net.conv1.weight.grad)

conv1 grad before backprop
None
conv1 grad after backprop
tensor([[[[ 7.8256e-03, -1.2072e-03, -6.7794e-03, -1.0931e-02,  5.9057e-03],
          [-3.6541e-03, -2.0363e-02,  1.1442e-02, -6.7667e-03,  1.1405e-02],
          [ 1.2727e-02,  1.6590e-02,  2.0165e-03, -3.7445e-03,  2.3650e-02],
          [ 1.5314e-03, -1.6798e-02,  1.2756e-02, -1.8046e-03, -1.3962e-02],
          [ 7.4986e-03, -9.4742e-03,  2.2131e-03,  6.4103e-03,  2.6853e-02]]],


        [[[-5.5541e-03, -8.4234e-03, -2.2605e-02, -6.0791e-03,  9.8344e-03],
          [-1.1196e-02,  1.4949e-02,  7.3339e-03, -1.6946e-02, -2.5760e-03],
          [-4.3093e-03,  3.5342e-04, -1.7460e-02,  3.1444e-02, -1.6968e-02],
          [ 2.1975e-03,  1.8721e-02,  2.4375e-02, -6.5204e-03, -1.5079e-02],
          [-4.6113e-03,  4.8170e-05, -1.2978e-03, -6.7232e-03,  1.4883e-02]]],


        [[[-1.5062e-02,  1.4291e-02,  3.0478e-03, -4.0698e-03, -9.4697e-03],
          [-2.8825e-03,  2.2096e-02,  3.7767e-03,  3.2314e-03,  2.6987e-03],
          

In [0]:
lr = 0.003
for param in net.parameters():
  param.data.sub_(param.grad.data * lr)

In [0]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.003)

#Training loop:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target) 
loss.backward()
optimizer.step()

In [62]:
print('conv1 grad after SGD backprop')
print(net.conv1.weight.grad)

conv1 grad after SGD backprop
tensor([[[[ 7.7669e-03, -1.2543e-03, -6.7836e-03, -1.0715e-02,  5.9997e-03],
          [-3.7381e-03, -2.0054e-02,  1.1414e-02, -6.7713e-03,  1.1561e-02],
          [ 1.2980e-02,  1.6440e-02,  2.1325e-03, -3.5988e-03,  2.3671e-02],
          [ 1.5571e-03, -1.6780e-02,  1.2497e-02, -1.6615e-03, -1.4283e-02],
          [ 7.3560e-03, -9.5862e-03,  2.0173e-03,  6.2795e-03,  2.7221e-02]]],


        [[[-5.4927e-03, -8.4265e-03, -2.2298e-02, -6.0706e-03,  9.7308e-03],
          [-1.1127e-02,  1.5295e-02,  7.3066e-03, -1.6987e-02, -2.5105e-03],
          [-3.9256e-03,  4.3962e-04, -1.7427e-02,  3.1461e-02, -1.7096e-02],
          [ 2.0151e-03,  1.8796e-02,  2.4462e-02, -6.4947e-03, -1.5197e-02],
          [-4.5610e-03, -3.9045e-04, -1.4335e-03, -7.0181e-03,  1.4520e-02]]],


        [[[-1.5372e-02,  1.4035e-02,  2.8614e-03, -3.9151e-03, -9.1778e-03],
          [-3.2690e-03,  2.2555e-02,  3.3456e-03,  3.8937e-03,  2.8667e-03],
          [ 6.9712e-03, -7.2250e-03, -